In [6]:
!pip install lxml

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   -------------------------------------- - 3.7/3.8 MB 24.3 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 20.6 MB/s eta 0:00:00


In [2]:
!pip install PyPDF2

In [9]:
import requests
from bs4 import BeautifulSoup

# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/Pittsburgh"

# Send a GET request
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)

# Parse the HTML content
soup = BeautifulSoup(response.text, "html.parser")

# Extract the title
title = soup.find("h1", {"id": "firstHeading"}).text.strip()

# Extract all section headings
headings = []
for heading in soup.find_all(["h2", "h3", "h4"]):
    span = heading.find("span", {"class": "mw-headline"})
    if span:
        headings.append(span.text.strip())

# Extract all paragraph text
paragraphs = []
for p in soup.find_all("p"):
    text = p.get_text().strip()
    if text:
        paragraphs.append(text)



# Extract lists (ul/li elements)
lists = []
for ul in soup.find_all("ul"):
    list_items = [li.get_text().strip() for li in ul.find_all("li")]
    if list_items:
        lists.append("\n".join(list_items))

# Extract tables and format them as readable text
tables = []
for table in soup.find_all("table", {"class": "wikitable"}):
    df = pd.read_html(str(table))[0]  # Convert table to DataFrame
    table_text = df.to_string(index=False)  # Convert to plain text
    tables.append(table_text)


# Extract references
references = []
for ref in soup.find_all("cite", {"class": "citation"}):
    link = ref.find("a", href=True)
    if link:
        references.append(link["href"])

# Save extracted content in a text file
filename = "pittsburgh_wikipedia.txt"
with open(filename, "w", encoding="utf-8") as f:
    f.write(f"Title: {title}\n\n")
    f.write("Headings:\n")
    f.write("\n".join(headings) + "\n\n")
    f.write("Content:\n")
    f.write("\n".join(paragraphs) + "\n\n")
    f.write("Lists:\n")
    f.write("\n\n".join(lists) + "\n\n")
    f.write("Tables:\n")
    f.write("\n\n".join(tables) + "\n\n")
    f.write("References:\n")
    f.write("\n".join(references) + "\n")

print(f"Scraping completed. Data saved in '{filename}'")


Scraping completed. Data saved in 'pittsburgh_wikipedia.txt'


C:\Users\maitr\AppData\Local\Temp\ipykernel_21960\2859862488.py:43: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # Convert table to DataFrame
C:\Users\maitr\AppData\Local\Temp\ipykernel_21960\2859862488.py:43: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # Convert table to DataFrame
C:\Users\maitr\AppData\Local\Temp\ipykernel_21960\2859862488.py:43: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # Convert table to DataFrame
C:\Users\maitr\AppData\Local\Temp\ipykernel_21960\2859862488.py:43: FutureWarning: Passing lit

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import time

# Wikipedia base URL
BASE_URL = "https://en.wikipedia.org"

# Wikipedia page URL (Pittsburgh)
MAIN_PAGE = "/wiki/Pittsburgh"

# Headers to avoid bot detection
HEADERS = {"User-Agent": "Mozilla/5.0"}

# Function to scrape a Wikipedia page and return its structured text
def scrape_wikipedia_page(relative_url):
    url = BASE_URL + relative_url
    response = requests.get(url, headers=HEADERS)
    
    if response.status_code != 200:
        print(f"Skipping {url} (Status Code: {response.status_code})")
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    
    # Extract page title
    title = soup.find("h1", {"id": "firstHeading"}).text.strip()
    
    content_list = [f"Title: {title}\n\n"]

    # Process the main content section
    for tag in soup.find_all(["h2", "h3", "h4", "p", "ul", "table"]):
        if tag.name in ["h2", "h3", "h4"]:  # Headings
            span = tag.find("span", {"class": "mw-headline"})
            if span:
                content_list.append(f"\n{span.text.strip()}\n" + "-" * len(span.text.strip()) + "\n")

        elif tag.name == "p":  # Paragraphs
            text = tag.get_text().strip()
            if text:
                content_list.append(text)

        elif tag.name == "ul":  # Lists
            list_items = [f"- {li.get_text().strip()}" for li in tag.find_all("li")]
            if list_items:
                content_list.append("\n".join(list_items))

        elif tag.name == "table" and "wikitable" in tag.get("class", []):  # Tables
            try:
                table_html = str(tag)
                table_df = pd.read_html(StringIO(table_html))[0]  # Use StringIO to fix warning
                table_text = table_df.to_string(index=False)  # Convert to plain text
                content_list.append("\nTable:\n" + table_text)
            except Exception as e:
                print(f"Skipping table in {url} due to error: {e}")

    return "\n\n".join(content_list)

# Scrape the main Pittsburgh page
print(f"Scraping main page: {MAIN_PAGE}")
pittsburgh_content = scrape_wikipedia_page(MAIN_PAGE)

# Extract all internal Wikipedia links from the main content section
soup = BeautifulSoup(requests.get(BASE_URL + MAIN_PAGE, headers=HEADERS).text, "html.parser")
internal_links = set()

for link in soup.find("div", {"id": "bodyContent"}).find_all("a", href=True):
    href = link["href"]
    if href.startswith("/wiki/") and ":" not in href:  # Exclude special Wikipedia pages
        internal_links.add(href)

# Only scrape **linked Wikipedia pages** that were found inside paragraphs
linked_pages_content = []

print(f"\nFound {len(internal_links)} internal links. Scraping all linked pages from paragraphs...\n")

for i, link in enumerate(internal_links):
    print(f"Scraping linked page {i+1}/{len(internal_links)}: {link}")
    page_content = scrape_wikipedia_page(link)
    if page_content:
        linked_pages_content.append(page_content)
    time.sleep(1)  # Delay to prevent Wikipedia from blocking requests

# Save extracted content in a text file
filename = "pittsburgh_wikipedia_with_links.txt"
with open(filename, "w", encoding="utf-8") as f:
    f.write(f"{pittsburgh_content}\n\n")
    f.write("\n\n".join(linked_pages_content))

print(f"\nScraping completed. Data saved in '{filename}'")


Scraping main page: /wiki/Pittsburgh

Found 1881 internal links. Scraping all linked pages from paragraphs...

Scraping linked page 1/1881: /wiki/Pennsylvania%27s_42nd_Senatorial_District
Scraping linked page 2/1881: /wiki/Rockford_metropolitan_area,_Illinois
Scraping linked page 3/1881: /wiki/Shane_Douglas
Scraping linked page 4/1881: /wiki/ISSN_(identifier)


KeyboardInterrupt: 

In [5]:
import requests
import PyPDF2
import os
from io import BytesIO

# List of PDF URLs
PDF_URLS = [
    "https://apps.pittsburghpa.gov/redtail/images/23255_2024_Operating_Budget.pdf",
    "https://www.pittsburghpa.gov/files/assets/city/v/1/finance/documents/tax-forms/9622_amusement_tax_regulations.pdf",
    "https://www.pittsburghpa.gov/files/assets/city/v/1/finance/documents/tax-forms/9626_payroll_tax_regulations.pdf",
    "https://www.pittsburghpa.gov/files/assets/city/v/1/finance/documents/tax-forms/9623_isp_tax_regulations.pdf",
    "https://www.pittsburghpa.gov/files/assets/city/v/1/finance/documents/tax-forms/9624_local_services_tax_regulations.pdf",
    "https://www.pittsburghpa.gov/files/assets/city/v/1/finance/documents/tax-forms/9625_parking_tax_regulations.pdf",
    "https://www.pittsburghpa.gov/files/assets/city/v/1/finance/documents/tax-forms/9627_uf_regulations.pdf"
]

# Output file
DATA_FILE = "pittsburgh_regulations.txt"
ERROR_LOG = "scraping_errors.log"

# Function to download and extract text from a PDF
def extract_text_from_pdf(url):
    """Downloads a PDF from the URL and extracts text."""
    try:
        response = requests.get(url, stream=True, timeout=15)  # 15s timeout
        if response.status_code != 200:
            print(f"Skipping {url} (Status Code: {response.status_code})")
            with open(ERROR_LOG, "a", encoding="utf-8") as err_file:
                err_file.write(f"Failed: {url} (Status Code: {response.status_code})\n")
            return None

        # Read PDF content
        pdf_file = BytesIO(response.content)
        reader = PyPDF2.PdfReader(pdf_file)

        # Extract text from all pages
        extracted_text = []
        for page in reader.pages:
            extracted_text.append(page.extract_text())

        pdf_text = "\n".join(filter(None, extracted_text))  # Remove None values

        return pdf_text

    except requests.exceptions.RequestException as e:
        print(f"Skipping {url} due to error: {e}")
        with open(ERROR_LOG, "a", encoding="utf-8") as err_file:
            err_file.write(f"Error: {url} ({e})\n")
        return None

# Function to process and save extracted PDF text
def scrape_pdfs(pdf_urls):
    """Scrapes all PDFs and writes extracted text into a file."""
    with open(DATA_FILE, "a", encoding="utf-8") as f:  # Append mode
        for i, url in enumerate(pdf_urls):
            print(f"Processing PDF {i+1}/{len(pdf_urls)}: {url}")
            pdf_text = extract_text_from_pdf(url)

            if pdf_text:
                # Extract title from URL
                title = os.path.basename(url).replace("_", " ").replace(".pdf", "").title()

                # Save extracted content
                f.write(f"\nTitle: {title}\nURL: {url}\n")
                f.write(f"{pdf_text}\n\n")
                f.flush()  # Ensure content is saved immediately

    print("\nPDF scraping completed. Data saved in", DATA_FILE)

# Run the PDF scraping
scrape_pdfs(PDF_URLS)


Processing PDF 1/7: https://apps.pittsburghpa.gov/redtail/images/23255_2024_Operating_Budget.pdf
Skipping https://apps.pittsburghpa.gov/redtail/images/23255_2024_Operating_Budget.pdf due to error: HTTPSConnectionPool(host='apps.pittsburghpa.gov', port=443): Max retries exceeded with url: /redtail/images/23255_2024_Operating_Budget.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)')))
Processing PDF 2/7: https://www.pittsburghpa.gov/files/assets/city/v/1/finance/documents/tax-forms/9622_amusement_tax_regulations.pdf
Processing PDF 3/7: https://www.pittsburghpa.gov/files/assets/city/v/1/finance/documents/tax-forms/9626_payroll_tax_regulations.pdf
Processing PDF 4/7: https://www.pittsburghpa.gov/files/assets/city/v/1/finance/documents/tax-forms/9623_isp_tax_regulations.pdf
Processing PDF 5/7: https://www.pittsburghpa.gov/files/assets/city/v/1/finance/documents/tax-forms/9624_

## Scrapping CMU Sites 

In [6]:
import requests
from bs4 import BeautifulSoup
import time

# Base domain
BASE_URL = "https://www.cmu.edu"

# Sitemap URL
START_URL = "https://www.cmu.edu/sitemap/"

# Headers to mimic a real browser request
HEADERS = {"User-Agent": "Mozilla/5.0"}

# Output files
DATA_FILE = "cmu_scraped_data.txt"
ERROR_LOG = "scraping_errors.log"

# Visited URLs (to prevent duplicate scraping)
visited_urls = set()

# Function to scrape a single webpage
def scrape_page(url, depth):
    """Scrapes headings and paragraphs from a given webpage."""
    if url in visited_urls or depth > 5:
        return None  # Avoid re-scraping and exceeding depth

    try:
        response = requests.get(url, headers=HEADERS, timeout=10)  # 10s timeout
        if response.status_code != 200:
            print(f"Skipping {url} (Status Code: {response.status_code})")
            with open(ERROR_LOG, "a", encoding="utf-8") as err_file:
                err_file.write(f"Failed: {url} (Status Code: {response.status_code})\n")
            return None

        visited_urls.add(url)  # Mark as visited
        soup = BeautifulSoup(response.text, "html.parser")
        title_tag = soup.find("title")
        title = title_tag.text.strip() if title_tag else "No Title"

        content_list = [f"Title: {title}\nURL: {url}\n"]

        # Extract headings & paragraphs
        for tag in soup.find_all(["h1", "h2", "h3", "p"]):
            if tag.name in ["h1", "h2", "h3"]:  # Headings
                content_list.append(f"\n{tag.text.strip()}\n" + "-" * len(tag.text.strip()) + "\n")
            elif tag.name == "p":  # Paragraphs
                text = tag.get_text().strip()
                if text:
                    content_list.append(text)

        # Write extracted content immediately
        with open(DATA_FILE, "a", encoding="utf-8") as f:
            f.write("\n".join(content_list) + "\n\n")

        return soup  # Return parsed HTML to extract links

    except requests.exceptions.RequestException as e:
        print(f"Skipping {url} due to error: {e}")
        with open(ERROR_LOG, "a", encoding="utf-8") as err_file:
            err_file.write(f"Error: {url} ({e})\n")
        return None

# Recursive function to scrape links up to depth 5
def recursive_scrape(url, depth):
    """Recursively scrapes a webpage and follows links up to depth 5."""
    if depth > 5 or url in visited_urls:
        return

    print(f"Scraping (Depth {depth}): {url}")
    soup = scrape_page(url, depth)

    if not soup:
        return

    # Extract all links and filter only CMU internal links
    for link in soup.find_all("a", href=True):
        href = link["href"]
        if href.startswith("/"):  # Internal relative link
            full_url = BASE_URL + href
            recursive_scrape(full_url, depth + 1)
        elif href.startswith(BASE_URL):  # Internal absolute link
            recursive_scrape(href, depth + 1)

    time.sleep(1)  # Polite scraping delay

# Start recursive scraping from the sitemap
recursive_scrape(START_URL, depth=1)
print("\nScraping completed. Data saved in", DATA_FILE)


Scraping (Depth 1): https://www.cmu.edu/sitemap/
Scraping (Depth 2): https://www.cmu.edu//www.cmu.edu/
Skipping https://www.cmu.edu//www.cmu.edu/ (Status Code: 404)
Scraping (Depth 2): https://www.cmu.edu
Scraping (Depth 3): https://www.cmu.edu//www.cmu.edu/
Skipping https://www.cmu.edu//www.cmu.edu/ (Status Code: 404)
Scraping (Depth 3): https://www.cmu.edu/news/stories/archives/2025/march/women-in-science-highlight-mentorship-outreach-as-integral-to-career-success
Scraping (Depth 4): https://www.cmu.edu/news/
Scraping (Depth 5): https://www.cmu.edu/news/stories
Scraping (Depth 5): https://www.cmu.edu/news/stories/college-of-engineering
Scraping (Depth 5): https://www.cmu.edu/news/stories/college-of-fine-arts
Scraping (Depth 5): https://www.cmu.edu/news/stories/dietrich-college-of-humanities-social-sciences
Scraping (Depth 5): https://www.cmu.edu/news/stories/heinz-college-of-information-systems-and-public-policy
Scraping (Depth 5): https://www.cmu.edu/news/stories/mellon-college-of-s

C:\Users\maitr\AppData\Local\Temp\ipykernel_22628\2773242819.py:36: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(response.text, "html.parser")


Scraping (Depth 5): https://www.cmu.edu/social-media/
Scraping (Depth 4): https://www.cmu.edu/dietrich/statistics-datascience/people/faculty/rebecca-nugent.html
Scraping (Depth 5): https://www.cmu.edu//www.cmu.edu/
Skipping https://www.cmu.edu//www.cmu.edu/ (Status Code: 404)
Scraping (Depth 5): https://www.cmu.edu/dietrich/
Scraping (Depth 5): https://www.cmu.edu//www.cmu.edu/legal/
Skipping https://www.cmu.edu//www.cmu.edu/legal/ (Status Code: 404)
Scraping (Depth 5): https://www.cmu.edu//www.cmu.edu/
Skipping https://www.cmu.edu//www.cmu.edu/ (Status Code: 404)
Scraping (Depth 5): https://www.cmu.edu/dietrich/statistics-datascience/rss-feeds/news.rss
Scraping (Depth 5): https://www.cmu.edu/career/outcomes/post-grad-dashboard.html
Scraping (Depth 5): https://www.cmu.edu/online/fds/index.html
Scraping (Depth 5): https://www.cmu.edu/online/fds/
Scraping (Depth 5): https://www.cmu.edu/dietrich/statistics-datascience/engagement/summer/index.html
Scraping (Depth 4): https://www.cmu.edu/ch

In [7]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin

# Base domain
BASE_URL = "https://www.cmu.edu"

# Sitemap URL
START_URL = "https://www.cmu.edu/sitemap/"

# Headers to mimic a real browser request
HEADERS = {"User-Agent": "Mozilla/5.0"}

# Output files
DATA_FILE = "cmu_scraped_data.txt"
ERROR_LOG = "scraping_errors.log"

# Visited URLs (to prevent duplicate scraping)
visited_urls = set()

# Function to extract all links from a page, handling relative URLs
def extract_links(soup):
    """Extracts all internal CMU links from a BeautifulSoup object."""
    links = set()
    for link in soup.find_all("a", href=True):
        href = link["href"]
        if href.startswith("/"):  # Relative link (convert to absolute)
            full_url = urljoin(BASE_URL, href)
        elif href.startswith("http") and "cmu.edu" in href:  # Absolute CMU link
            full_url = href
        else:
            continue  # Ignore external links

        links.add(full_url)
    return links

# Function to scrape a single webpage
def scrape_page(url, depth):
    """Scrapes headings and paragraphs from a given webpage."""
    if url in visited_urls or depth > 5:
        return None  # Avoid re-scraping and exceeding depth

    try:
        response = requests.get(url, headers=HEADERS, timeout=10)  # 10s timeout
        if response.status_code != 200:
            print(f"Skipping {url} (Status Code: {response.status_code})")
            with open(ERROR_LOG, "a", encoding="utf-8") as err_file:
                err_file.write(f"Failed: {url} (Status Code: {response.status_code})\n")
            return None

        visited_urls.add(url)  # Mark as visited
        soup = BeautifulSoup(response.text, "html.parser")
        title_tag = soup.find("title")
        title = title_tag.text.strip() if title_tag else "No Title"

        content_list = [f"Title: {title}\nURL: {url}\n"]

        # Extract headings & paragraphs
        for tag in soup.find_all(["h1", "h2", "h3", "p"]):
            if tag.name in ["h1", "h2", "h3"]:  # Headings
                content_list.append(f"\n{tag.text.strip()}\n" + "-" * len(tag.text.strip()) + "\n")
            elif tag.name == "p":  # Paragraphs
                text = tag.get_text().strip()
                if text:
                    content_list.append(text)

        # Write extracted content immediately
        with open(DATA_FILE, "a", encoding="utf-8") as f:
            f.write("\n".join(content_list) + "\n\n")

        return soup  # Return parsed HTML to extract links

    except requests.exceptions.RequestException as e:
        print(f"Skipping {url} due to error: {e}")
        with open(ERROR_LOG, "a", encoding="utf-8") as err_file:
            err_file.write(f"Error: {url} ({e})\n")
        return None

# Recursive function to scrape links up to depth 5
def recursive_scrape(url, depth):
    """Recursively scrapes a webpage and follows links up to depth 5."""
    if depth > 5 or url in visited_urls:
        return

    print(f"Scraping (Depth {depth}): {url}")
    soup = scrape_page(url, depth)

    if not soup:
        return

    # Extract all valid CMU links from the page
    page_links = extract_links(soup)

    # Follow each extracted link recursively
    for link in page_links:
        recursive_scrape(link, depth + 1)

    time.sleep(1)  # Polite scraping delay

# Start recursive scraping from the sitemap
recursive_scrape(START_URL, depth=1)
print("\nScraping completed. Data saved in", DATA_FILE)


Scraping (Depth 1): https://www.cmu.edu/sitemap/
Scraping (Depth 2): https://www.cmu.edu/news/
Scraping (Depth 3): https://www.cmu.edu/news/stories/archives/2022/march
Scraping (Depth 4): https://www.cmu.edu/news/stories/archives/2013/february
Scraping (Depth 5): http://www.cmu.edu/news/feeds/news.rss


C:\Users\maitr\AppData\Local\Temp\ipykernel_22628\3388348632.py:53: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(response.text, "html.parser")


Scraping (Depth 5): http://www.cmu.edu/events
Scraping (Depth 5): https://www.cmu.edu/legal/
Scraping (Depth 5): https://www.cmu.edu/
Scraping (Depth 5): http://www.cmu.edu/social-media/
Scraping (Depth 5): http://www.cmu.edu/piper/
Scraping (Depth 4): https://www.cmu.edu/news/stories/archives/2013/january
Scraping (Depth 4): https://www.cmu.edu/news/stories/archives/2019/december
Scraping (Depth 4): https://www.cmu.edu/news/stories/archives/2011/march
Scraping (Depth 4): https://www.cmu.edu/news/stories/archives/2022/march/cmu-building-moonshot-mission-control-for-upcoming-lunar-exploration
Scraping (Depth 5): https://x.com/intent/tweet?url=https%3A%2F%2Fwww.cmu.edu%2Fnews%2Fstories%2Farchives%2F2022%2Fmarch%2Fcmu-building-moonshot-mission-control-for-upcoming-lunar-exploration&text=CMU%20Building%20Moonshot%20Mission%20Control%20for%20Upcoming%20Lunar%20Exploration
Skipping https://x.com/intent/tweet?url=https%3A%2F%2Fwww.cmu.edu%2Fnews%2Fstories%2Farchives%2F2022%2Fmarch%2Fcmu-build